# 1. Imports

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@10c759cb

In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._



In [3]:
def sc = spark.sparkContext
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

defined function sc
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@74779765

In [4]:
import spark.implicits._
// val df = sqlContext.read.parquet("data/test/part-00000-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet")
// df.count()

import spark.implicits._
// val df = sqlContext.read.parquet("data/test/part-00000-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet")
// df.count()

In [5]:
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler, MinMaxScaler}
import org.apache.spark.ml.regression.{GBTRegressor, LinearRegression, RandomForestRegressor}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.RegressionEvaluator

import $ivy.$                                    


import org.apache.spark.ml.linalg.{Matrix, Vectors}

import org.apache.spark.ml.stat.Correlation

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler, MinMaxScaler}

import org.apache.spark.ml.regression.{GBTRegressor, LinearRegression, RandomForestRegressor}

import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.ml.evaluation.RegressionEvaluator

# 2. Read Data

In [6]:
val train_df = sqlContext.read.parquet("data/train/")

parquet at cmd5.sc:1

1 / 1

train_df: DataFrame = [index: bigint, date_time: timestamp ... 103 more fields]

In [7]:
val test_df = sqlContext.read.parquet("data/test/")

parquet at cmd6.sc:1

1 / 1

test_df: DataFrame = [index: bigint, date_time: timestamp ... 103 more fields]

# 3. Fast EDA

In [8]:
train_df.printSchema

root
 |-- index: long (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- T_data_1_1: long (nullable = true)
 |-- T_data_1_2: long (nullable = true)
 |-- T_data_1_3: long (nullable = true)
 |-- T_data_2_1: long (nullable = true)
 |-- T_data_2_2: long (nullable = true)
 |-- T_data_2_3: long (nullable = true)
 |-- T_data_3_1: long (nullable = true)
 |-- T_data_3_2: long (nullable = true)
 |-- T_data_3_3: long (nullable = true)
 |-- T_data_4_1: long (nullable = true)
 |-- T_data_4_2: long (nullable = true)
 |-- T_data_4_3: long (nullable = true)
 |-- T_data_5_1: long (nullable = true)
 |-- T_data_5_2: long (nullable = true)
 |-- T_data_5_3: long (nullable = true)
 |-- H_data: double (nullable = true)
 |-- AH_data: double (nullable = true)
 |-- T_data_1_1_1: long (nullable = true)
 |-- T_data_1_2_1: long (nullable = true)
 |-- T_data_1_3_1: long (nullable = true)
 |-- T_data_2_1_1: long (nullable = true)
 |-- T_data_2_2_1: long (nullable = true)
 |-- T_data_2_3_1: long (null

### 3.1 Train / Test shapes

##### Train

In [9]:
val n_rows = train_df.count()

count at cmd8.sc:1

6 / 6

count at cmd8.sc:1

1 / 1

n_rows: Long = 26232L

In [10]:
val n_columns = train_df.columns.length

n_columns: Int = 105

##### Test

In [11]:
val n_rows = test_df.count()

count at cmd10.sc:1

1 / 1

count at cmd10.sc:1

1 / 1

n_rows: Long = 2952L

In [12]:
val n_columns = test_df.columns.length

n_columns: Int = 105

### 3.2 Train / Test targets distribution

##### Train

In [13]:
train_df.select("quality").describe().show()

describe at cmd12.sc:1

6 / 6

describe at cmd12.sc:1

1 / 1

+-------+-----------------+
|summary|          quality|
+-------+-----------------+
|  count|            26232|
|   mean|403.0612229338213|
| stddev|46.10377250338848|
|    min|              221|
|    max|              505|
+-------+-----------------+



##### Test

In [14]:
test_df.select("quality").describe().show()

describe at cmd13.sc:1

1 / 1

describe at cmd13.sc:1

1 / 1

+-------+-----------------+
|summary|          quality|
+-------+-----------------+
|  count|             2952|
|   mean|400.4861111111111|
| stddev|   47.69820489038|
|    min|              226|
|    max|              493|
+-------+-----------------+



# 4. Select features for model

In [15]:
val feature_columns = scala.collection.mutable.ListBuffer[String]()
for (elem <- train_df.schema) {
    if ((elem.name.toString != "index") & (elem.name.toString != "date_time"))
    
    feature_columns += elem.name.toString
}

feature_columns: collection.mutable.ListBuffer[String] = ListBuffer(
  "T_data_1_1",
...

In [16]:
feature_columns.length

res15: Int = 103

# 5. Vector assembler

In [17]:
val assembler = new VectorAssembler().setInputCols(feature_columns.filter(_ != "quality").toArray).setOutputCol("features")
val output_train = assembler.transform(train_df).select("features", "quality")
val output_test = assembler.transform(test_df).select("features", "quality")

assembler: VectorAssembler = vecAssembler_dba5244c6c2d
output_train: DataFrame = [features: vector, quality: bigint]
output_test: DataFrame = [features: vector, quality: bigint]

In [18]:
output_train.select("features", "quality").show(5)

show at cmd17.sc:1

1 / 1

+--------------------+-------+
|            features|quality|
+--------------------+-------+
|[200.0,206.0,208....|    365|
|[206.0,203.0,209....|    372|
|[198.0,204.0,207....|    360|
|[203.0,205.0,199....|    344|
|[207.0,205.0,197....|    336|
+--------------------+-------+
only showing top 5 rows



In [19]:
output_test.select("features", "quality").show(5)

show at cmd18.sc:1

1 / 1

+--------------------+-------+
|            features|quality|
+--------------------+-------+
|[281.0,267.0,274....|    441|
|[280.0,269.0,291....|    449|
|[280.0,280.0,270....|    449|
|[278.0,289.0,266....|    445|
|[284.0,292.0,279....|    442|
+--------------------+-------+
only showing top 5 rows



# 6. Scaler

In [20]:
val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("Scaled_features")
val output_scaled_train = scaler.fit(output_train).transform(output_train)
val output_scaled_test = scaler.fit(output_test).transform(output_test)

treeAggregate at RowMatrix.scala:433

6 / 6

treeAggregate at RowMatrix.scala:433

2 / 2

treeAggregate at RowMatrix.scala:433

1 / 1

scaler: MinMaxScaler = minMaxScal_c0ae1d44ec59
output_scaled_train: DataFrame = [features: vector, quality: bigint ... 1 more field]
output_scaled_test: DataFrame = [features: vector, quality: bigint ... 1 more field]

In [21]:
output_scaled_train.select("features","Scaled_features").show(5)

show at cmd20.sc:1

1 / 1

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|[200.0,206.0,208....|[0.42950819672131...|
|[206.0,203.0,209....|[0.43606557377049...|
|[198.0,204.0,207....|[0.42732240437158...|
|[203.0,205.0,199....|[0.43278688524590...|
|[207.0,205.0,197....|[0.43715846994535...|
+--------------------+--------------------+
only showing top 5 rows



In [22]:
output_scaled_test.select("features","Scaled_features").show(5)

show at cmd21.sc:1

1 / 1

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|[281.0,267.0,274....|[0.49249249249249...|
|[280.0,269.0,291....|[0.49099099099099...|
|[280.0,280.0,270....|[0.49099099099099...|
|[278.0,289.0,266....|[0.48798798798798...|
|[284.0,292.0,279....|[0.49699699699699...|
+--------------------+--------------------+
only showing top 5 rows



# 7. Split

In [23]:
val Array(training, validating) = output_scaled_train.randomSplit(Array(0.6, 0.4), seed = 777)

training: Dataset[Row] = [features: vector, quality: bigint ... 1 more field]
validating: Dataset[Row] = [features: vector, quality: bigint ... 1 more field]

# 8. Linear regression model

In [24]:
// Extract the summary from the returned LogisticRegressionModel instance trained in the earlier
val lr = new LinearRegression()
.setMaxIter(45)
.setRegParam(0.02)
.setElasticNetParam(0.08)
.setFeaturesCol("Scaled_features")
.setLabelCol("quality")


val model = lr.fit(training)
val predict_train_lr = model.transform(training)
val predict_val_lr = model.transform(validating)
val predict_test_lr = model.transform(output_scaled_test)
//predict_test.select("Outcome","prediction").show(10)

first at LinearRegression.scala:322

1 / 1

treeAggregate at WeightedLeastSquares.scala:105

6 / 6

21/01/23 11:32:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/01/23 11:32:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


...

count at LinearRegression.scala:953

6 / 6

count at LinearRegression.scala:953

1 / 1

lr: LinearRegression = linReg_93bff69e56b2
model: org.apache.spark.ml.regression.LinearRegressionModel = linReg_93bff69e56b2
predict_train_lr: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_val_lr: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_test_lr: DataFrame = [features: vector, quality: bigint ... 2 more fields]

In [25]:
predict_test_lr.show(5)

show at cmd24.sc:1

1 / 1

+--------------------+-------+--------------------+------------------+
|            features|quality|     Scaled_features|        prediction|
+--------------------+-------+--------------------+------------------+
|[281.0,267.0,274....|    441|[0.49249249249249...|409.54021723674197|
|[280.0,269.0,291....|    449|[0.49099099099099...|423.14623841384855|
|[280.0,280.0,270....|    449|[0.49099099099099...| 436.5579493047248|
|[278.0,289.0,266....|    445|[0.48798798798798...| 414.5005016131549|
|[284.0,292.0,279....|    442|[0.49699699699699...|389.84763539110867|
+--------------------+-------+--------------------+------------------+
only showing top 5 rows



In [26]:
val evaluator = new RegressionEvaluator()
.setPredictionCol("prediction")
.setLabelCol("quality")

val lr_auc_train = evaluator.evaluate(predict_train_lr)
val lr_auc_val = evaluator.evaluate(predict_val_lr)
val lr_auc_test = evaluator.evaluate(predict_test_lr)

treeAggregate at RegressionMetrics.scala:57

6 / 6

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

...

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

1 / 1

evaluator: RegressionEvaluator = regEval_432daefff119
lr_auc_train: Double = 22.71900783154007
lr_auc_val: Double = 23.444787629455753
lr_auc_test: Double = 79.15769966712071

# 9. Random forest model

In [27]:
val rf = new RandomForestRegressor()
.setFeaturesCol("Scaled_features")
.setLabelCol("quality")


val model = rf.fit(training)
val predict_train_rf = model.transform(training)
val predict_val_rf = model.transform(validating)
val predict_test_rf = model.transform(output_scaled_test)

take at DecisionTreeMetadata.scala:112

1 / 1

count at DecisionTreeMetadata.scala:118

6 / 6

flatMap at RandomForest.scala:919

...

collectAsMap at RandomForest.scala:557

6 / 6

first at RandomForestRegressor.scala:136

1 / 1

rf: RandomForestRegressor = rfr_bac3e25af999
model: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_bac3e25af999) with 20 trees
predict_train_rf: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_val_rf: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_test_rf: DataFrame = [features: vector, quality: bigint ... 2 more fields]

In [28]:
predict_test_rf.show(5)

show at cmd27.sc:1

1 / 1

+--------------------+-------+--------------------+------------------+
|            features|quality|     Scaled_features|        prediction|
+--------------------+-------+--------------------+------------------+
|[281.0,267.0,274....|    441|[0.49249249249249...|435.44879881620426|
|[280.0,269.0,291....|    449|[0.49099099099099...|  439.326145312555|
|[280.0,280.0,270....|    449|[0.49099099099099...| 442.4433028598426|
|[278.0,289.0,266....|    445|[0.48798798798798...| 446.9106061311926|
|[284.0,292.0,279....|    442|[0.49699699699699...|443.36116302469225|
+--------------------+-------+--------------------+------------------+
only showing top 5 rows



In [29]:
val evaluator = new RegressionEvaluator()
.setPredictionCol("prediction")
.setLabelCol("quality")

val rf_auc_train = evaluator.evaluate(predict_train_rf)
val rf_auc_val = evaluator.evaluate(predict_val_rf)
val rf_auc_test = evaluator.evaluate(predict_test_rf)

treeAggregate at RegressionMetrics.scala:57

6 / 6

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

...

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

1 / 1

evaluator: RegressionEvaluator = regEval_093ecd580986
rf_auc_train: Double = 18.259107039146137
rf_auc_val: Double = 18.70799915618574
rf_auc_test: Double = 31.651548438316237

# 10. Gradient boosting model

In [30]:
val gb = new GBTRegressor()
.setFeaturesCol("Scaled_features")
.setLabelCol("quality")


val model = gb.fit(training)
val predict_train_gb = model.transform(training)
val predict_val_gb = model.transform(validating)
val predict_test_gb = model.transform(output_scaled_test)

first at GBTRegressor.scala:169

1 / 1

take at DecisionTreeMetadata.scala:112

1 / 1

count at DecisionTreeMetadata.scala:118

...

mapPartitions at RandomForest.scala:538

6 / 6

collectAsMap at RandomForest.scala:557

6 / 6

gb: GBTRegressor = gbtr_0274775d3814
model: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel (uid=gbtr_0274775d3814) with 20 trees
predict_train_gb: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_val_gb: DataFrame = [features: vector, quality: bigint ... 2 more fields]
predict_test_gb: DataFrame = [features: vector, quality: bigint ... 2 more fields]

In [31]:
predict_test_gb.show(5)

show at cmd30.sc:1

1 / 1

+--------------------+-------+--------------------+------------------+
|            features|quality|     Scaled_features|        prediction|
+--------------------+-------+--------------------+------------------+
|[281.0,267.0,274....|    441|[0.49249249249249...|457.52954072887223|
|[280.0,269.0,291....|    449|[0.49099099099099...|446.38732397656935|
|[280.0,280.0,270....|    449|[0.49099099099099...|439.74498240755713|
|[278.0,289.0,266....|    445|[0.48798798798798...|445.29847192854163|
|[284.0,292.0,279....|    442|[0.49699699699699...|438.74927214275044|
+--------------------+-------+--------------------+------------------+
only showing top 5 rows



In [32]:
val evaluator = new RegressionEvaluator()
.setPredictionCol("prediction")
.setLabelCol("quality")

val gb_auc_train = evaluator.evaluate(predict_train_gb)
val gb_auc_val = evaluator.evaluate(predict_val_gb)
val gb_auc_test = evaluator.evaluate(predict_test_gb)

treeAggregate at RegressionMetrics.scala:57

6 / 6

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

...

treeAggregate at RegressionMetrics.scala:57

2 / 2

treeAggregate at RegressionMetrics.scala:57

1 / 1

evaluator: RegressionEvaluator = regEval_4c3309420398
gb_auc_train: Double = 15.116248045903834
gb_auc_val: Double = 16.49689372801666
gb_auc_test: Double = 31.32625089608374